# FewShotPainAdaptation on Google Colab (T4)
This notebook installs dependencies, verifies GPU, configures reproducibility, and runs LOSO few-shot training.

Before running, set Colab runtime to: **GPU (T4)**.

In [1]:
!nvidia-smi
!pip -q install -U pip
!pip -q install tensorflow==2.18.1 cloudpickle matplotlib numpy pandas scikit-learn scipy seaborn pydantic

Fri Feb 27 16:23:56 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P0             30W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
REPO_URL = "https://github.com/hhihn/FewShotPainAdaptation.git"
PROJECT_DIR = "/content/FewShotPainAdaptation"
import os
if not os.path.isdir(PROJECT_DIR):
    !git clone $REPO_URL $PROJECT_DIR
    %cd $PROJECT_DIR
else:
    %cd $PROJECT_DIR
    !git pull


/content/FewShotPainAdaptation
Already up to date.


In [4]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print('Visible GPUs:', gpus)
if not gpus:
    raise RuntimeError('No GPU detected. In Colab, set Runtime -> Change runtime type -> GPU.') # Optional, helps speed on T4.
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print('Mixed precision policy:', mixed_precision.global_policy())

Visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Mixed precision policy: <DTypePolicy "mixed_float16">


In [5]:
# Point this to a folder containing: X_pre.npy, y_heater.npy, subjects.npy\n# Example if your data is in repo: /content/FewShotPainAdaptation/data\n# Example if in Drive: /content/drive/MyDrive/FewShotPainAdaptation/data\n
DATA_DIR = "/content/drive/MyDrive/PainData"
import os
required = ['X_pre.npy', 'y_heater.npy', 'subjects.npy']
missing = [f for f in required if not os.path.exists(os.path.join(DATA_DIR, f))]
if missing:
    raise FileNotFoundError(f'Missing files in DATA_DIR={DATA_DIR}: {missing}')
print('Using data directory:', DATA_DIR)

Using data directory: /content/drive/MyDrive/PainData


In [6]:
import json
import logging
import numpy as np
from data_loaders.pain_ds_config import PainDatasetConfig
from learner.few_shot_pain_learner import FewShotPainLearner
from utils.logger import setup_logger
logger = setup_logger('FewShotPainLearner', level=logging.INFO)
# Reproducible config
config = PainDatasetConfig(seed=42, deterministic_ops=True, k_shot=5, q_query=5,)
# Keep these small first; increase after sanity check.
NUM_EPOCHS = 3
EPISODES_PER_EPOCH = 500
VAL_EPISODES = 5
FUSION_METHODS = ['attention']
all_results = {}
for fusion_method in FUSION_METHODS:
    logger.info(f'Training with fusion method: {fusion_method}')
    learner = FewShotPainLearner(config=config,
                                 data_dir=DATA_DIR,
                                 learning_rate=1e-3,
                                 fusion_method=fusion_method,
                                 seed=config.seed,
                                 deterministic_ops=config.deterministic_ops)
    cv_results = learner.train(num_epochs=NUM_EPOCHS, episodes_per_epoch=EPISODES_PER_EPOCH, val_episodes=VAL_EPISODES,)
    all_results[fusion_method] = cv_results
    summary = {
        fm: {
            'avg_test_acc': float(np.mean(res['test_accuracies'])),
            'std_test_acc': float(np.std(res['test_accuracies'])),
            'avg_test_loss': float(np.mean(res['test_losses'])),
        }    for fm, res in all_results.items()}
    print(json.dumps(summary, indent=2))
    with open('colab_run_results.json', 'w') as f:
        json.dump({'summary': summary, 'full': all_results}, f, indent=2)
    print('Saved results to colab_run_results.json')

2026-02-27 16:24:27 │ INFO     │ FewShotPainLearner:17	 │ Training with fusion method: attention
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:72	 │ Loading data from /content/drive/MyDrive/PainData...
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:78	 │ X.shape: (2495, 2500, 3, 1)
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:80	 │ y_onehot.shape: (2495, 6)
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:82	 │ subjects.shape: [ 0  0  0 ... 51 51 51]
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:95	 │ Number of subjects: 52
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:96	 │   Data shape: (2495, 2500, 3)
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:97	 │   Labels shape: (2495,)
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:98	 │   Number of subjects: 52
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:99	 │   Samples per subject: ~47
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:100	 │   Classes: [0 1 2 3 4 5]
2026-02-27 16:24:27 │ INFO     │ PainMetaDataset:13

Model: "tcn_EDA"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tcn_block_0 (Functional)        │ (None, 2500, 32)       │         3,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_1 (Functional)        │ (None, 2500, 64)       │        21,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn_block_2 (Functional)        │ (None, 2500, 128)      │        83,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ self_attention                  │ ?                      │   0 (unbuilt) │
│ (MultiHeadAttention)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_pool                  │ ?                      │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_norm                  │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_pool                     │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_dense (Dense)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_norm                  │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 108,064 (422.12 KB)

 Trainable params: 107,168 (418.62 KB)

...

 Trainable params: 107,168 (418.62 KB)

 Non-trainable params: 896 (3.50 KB)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'multimodal_prototypical_network', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


2026-02-27 16:24:43 │ INFO     │ few_shot_pain_learner:67	 │ [Fold 1/52 | 1.9%] [Epoch 1/3 | 33.3%] [Train episode 1/500 | 0.2%] loss=1.7535, accuracy=0.3333
2026-02-27 16:25:03 │ INFO     │ few_shot_pain_learner:67	 │ [Fold 1/52 | 1.9%] [Epoch 1/3 | 33.3%] [Train episode 10/500 | 2.0%] loss=1.7895, accuracy=0.3333
2026-02-27 16:25:26 │ INFO     │ few_shot_pain_learner:67	 │ [Fold 1/52 | 1.9%] [Epoch 1/3 | 33.3%] [Train episode 20/500 | 4.0%] loss=1.8038, accuracy=0.1667
2026-02-27 16:25:47 │ INFO     │ few_shot_pain_learner:67	 │ [Fold 1/52 | 1.9%] [Epoch 1/3 | 33.3%] [Train episode 30/500 | 6.0%] loss=1.7138, accuracy=0.4444
2026-02-27 16:26:10 │ INFO     │ few_shot_pain_learner:67	 │ [Fold 1/52 | 1.9%] [Epoch 1/3 | 33.3%] [Train episode 40/500 | 8.0%] loss=1.7226, accuracy=0.2778
2026-02-27 16:26:30 │ INFO     │ few_shot_pain_learner:67	 │ [Fold 1/52 | 1.9%] [Epoch 1/3 | 33.3%] [Train episode 50/500 | 10.0%] loss=1.8141, accuracy=0.2222
2026-02-27 16:26:52 │ INFO     │ few_shot_pain

KeyboardInterrupt: 